# 使用激光测距仪测量材料折射率

## 实验原理
激光在不同折射率的材料中折射率不同，但激光测距仪仅会使用空气中光速来计算距离。通过比较材料置于测距路径时的测距值和空气中的测距值，可以计算出材料的折射率。

实验所用的材料为亚克力。

材料的折射率真值通过分光机（最小偏向角）测得，作为近似真值。


In [12]:
import numpy as np

def calculate_refractive_index(A, sigma_min):
    n = (np.sin(0.5 * (sigma_min + A))) / (np.sin(0.5 * A))
    return n
   

# convert degrees to radians 
A = np.radians(59 + 59/60) 
# purple, green, yellow
sigma_min_list = [np.radians(36 + 58/60),
                  np.radians(36 + 7/60),
                  np.radians(36)]

for sigma_min in sigma_min_list:
    print(calculate_refractive_index(A, sigma_min))

1.4977103678403934
1.4878317633125941
1.486469475056611


# 使用分光机测量亚克力的折射率

## 柯西色散公式与折射率修正

$$
n(\lambda) = B + \frac{C}{\lambda^2} + \frac{D}{\lambda^4}
$$

利用三组波长和折射率联立解得柯西色散公式系数

代入黄色光（波长为0.589微米）计算得到标准折射率。

In [10]:
from sympy import symbols, Eq, solve

def cal_cauchy_coe(n_list, wavelength_list):
    B, C, D = symbols('B C D')

    # define equations
    eq1 = Eq(B + C / wavelength_list[0]**2 + D / wavelength_list[0]**4, n_list[0])
    eq2 = Eq(B + C / wavelength_list[1]**2 + D / wavelength_list[1]**4, n_list[1])
    eq3 = Eq(B + C / wavelength_list[2]**2 + D / wavelength_list[2]**4, n_list[2])

    solution = solve((eq1, eq2, eq3), (B, C, D), list=True)
    return [solution[B], solution[C], solution[D]]

def cauchy_equation(wavelength, coes):
    B = coes[0]
    C = coes[1]
    D = coes[2]
    n = B + C / wavelength**2 + D / wavelength**4
    return n

# purple, green, yellow
n_list = [1.49771, 1.48783, 1.48647]
wave_list = [0.4358, 0.5461, 0.5790]

coes = cal_cauchy_coe(n_list, wave_list)
print(coes)

# substitute 0.589 to get the corrected refractive index
n_corrected = cauchy_equation(0.589, coes)

print(n_corrected)

[1.48205862056845, -0.000473228255535702, 0.000654425131973435]
1.48613203381719
